# Tutorial: Quantum ESPRESSO wrapper

In this tutorial we will go through a simple example of how to use the wrapper for the Quantum Espresso simulation engine.
You can find the wrapper [here](https://github.com/simphony/quantum-espresso-wrapper).

## Background
This is an example of a slightly different design based upon the input-output functionality of certain simulation engines such as Quantum Espresso and Gromacs.


## Let's get hands-on
### Installation
To run the local installation of Quantum Espresso, simply run `./install_engine.sh`. This should check for the prerequisites and compile the code for Quantum Espresso for you.

If the script runs into an error finding openmpi-bin or something like that, try running `apt-get update` and try again. 
Once the installation has completed, try running `pw.x` to see if the installation has succeeded. If this does not work, then try adding `export PATH=$PATH:/home/username/qe-6.1/bin/` at the end of `.bashrc` located at your home folder.  
  
Once you have verified that `pw.x` works, install the ontology via `pico install ontology.simlammps.yml`, and make sure to run `python3 setup.py` located in the root of the quantum espresso wrapper folder.   

That should be all needed to use Quantum Espresso!

### Simple example

This is an adaptation of quantum-espresso-wrapper/examples/Simple.py. As usual, we need to import the necessary components:

In [ ]:
import numpy as np 

from osp.core.namespaces import QE
from osp.core.utils import pretty_print
from osp.wrappers.quantumespresso.qe_session import qeSession

Next, we create simulation and its K points, which determine at what points it samples the cell

In [ ]:
sim = QE.Simulation()
k = QE.K_POINTS(vector = (7, 7, 7), unit = "")


Next, we create a cell, the element Silicon, a pseudopotential, an atom and the cell parameters. Note that the pseudopotential files should ALWAYS be located inside of a folder named `$PSEUDO_DIR` inside of wherever you are running the simulation.

In [ ]:
SiCell = QE.Cell()
Si = QE.Element(name = "Si")
SiPseudo = QE.PSEUDOPOTENTIAL(name = "Si.pbe-n-kjpaw_psl.1.0.0.UPF")
Si1 = QE.Atom()
SiParams = QE.CellParams()
celldm1 = QE.Celldm1(value = 5.43070, unit = "au")

Next, we connect these all to each other using the `add` method.

In [ ]:
Si.add(SiPseudo, Si1)
Si.add(QE.Mass(value = 28.085, unit = "amu"))
SiCell.add(Si1, SiParams)
Si1.add(QE.Position(vector = (0, 0, 0), unit = ""))
SiCell.add(celldm1)

We specify the cell parameters:

In [ ]:
SiParams.add(QE.CellParameterX(vector = (0.5, 0.5, 0), unit = ""),
             QE.CellParameterY(vector = (0.5, 0, 0.5), unit = ""),
             QE.CellParameterZ(vector = (0, 0.5, 0.5), unit = ""))

And then we add everything created so far to the simulation:


In [ ]:
sim.add(SiCell)
sim.add(Si)
sim.add(k)

While we're add it, let's add some variables to the simulation which we can check to see if they have been updated. They will not be taken into account when simulating, so they're there for control purposes.


In [ ]:
sim.add(QE.Pressure(value = 100, unit = "kbar"))
sim.add(QE.StressTensor(tensor2 = np.zeros((3, 3)), unit = "kbar"))

Let's check out what this simulation looks like now with the `pretty_print` function:

In [ ]:
pretty_print(sim)

Now, it's time to get the simulation running:

In [ ]:
session = qeSession(root = "")
quantum_espresso_wrapper = QE.QEWrapper(session = session)
quantum_espresso_wrapper.add(sim)
print("Running calculation...")

quantum_espresso_wrapper.session._run(prefix = "si", command_type = "pw.x", calculation_type = "scf")

Now let's check the results of our calculation:

In [ ]:
pretty_print(sim)

As you can see, the original part of the cuds tree is still there, with everything mostly the same. The new parts are:

- The qe.PwOut cuds object. This is the output file of the simulation, in case there is something that the wrapper does not parse but that you would still like to see.
- The qe.TotalEnergy cuds object. This was parsed from the qe.PwOut file itself.
- The qe.Force cuds object. This represents the force exerted on the atom(s).

The updated parts are:

- The qe.Pressure cuds object, having changed in value from 100 kbar to 5723.64 kbar.
- The qe.StressTensor cuds object, which is no longer zero.

Let's see if we can do better and calculate some bands structures:

In [ ]:
quantum_espresso_wrapper.session._run(prefix = "si", command_type = "pw.x", calculation_type = "bands")
quantum_espresso_wrapper.session._run(prefix = "si", command_type = "bands.x", calculation_type = "")

Although the cuds structure won't have changed much by this, the data is there in the folder.

Now let's try to relax this cell. While it isn't a real cell, we can still perform the calculations to relax it to know what the movement of the atoms would be like if it were a real cell (warning, perform vc-relax type calculations with caution. These examples are designed to be lightweight and non-indicative of real-world applications).

In [ ]:
quantum_espresso_wrapper.session._run(simulation = sim, prefix = "si", command_type = "pw.x", calculation_type = "relax", IONS = {'ion_dynamics': "'bfgs'"})

In [ ]:
pretty_print(sim)

In this example, the position hasn't changed. So let's spice things up a little bit and add another atom, and then relax:

In [ ]:
Si2 = QE.Atom()
Si2.add(QE.Position(vector = (0.25, 0.25, 0.26), unit = ""))
SiCell.add(Si)
Si.add(Si2)
pretty_print(sim)

In [ ]:
quantum_espresso_wrapper.session._run(simulation = sim, prefix = "si", command_type = "pw.x", calculation_type = "relax", IONS = {'ion_dynamics': "'bfgs'"})
pretty_print(sim)